# In-place FFT 

In [1]:
def permute_indices(N, R_seq):
    idx = [i for i in range(N)]
    P = 1
    for R in reversed(R_seq[1:]):
        g = N//P
        def map_fn(i):
            s = i // g
            i = i % g
            res = (i%R)* g//R + i//R + s * g
            return res
        for i in range(N):
            idx[i] = map_fn(idx[i])
        P *= R
    return idx

In [2]:
import numpy as np

def fft(X_, R_seq):
    X = np.zeros_like(X_, dtype=np.complex128)
    N = len(X)
    assert np.prod(R_seq) == N, "Product of radices must equal the length of the input array."

    # index reordering
    idx = permute_indices(N, R_seq)        
    for i in range(N):
        X[idx[i]] = X_[i]
                
    P = N
    for R in R_seq:
        P //= R
        T = N // R
        for i in range(T):
            # k, s = i // P, i % P
            k = i % (T//P)
            s = i // (T//P)
            idx = [k + r * N//(P*R) + s * N//P for r in range(R)]
            res = [0] * R
            # print(f'{i} : {idx} -> {idx}')
            for t in range(R):
                for r in range(R):
                    w0 = r*t/R
                    w1 = (r*k*P)/N
                    twiddle = np.exp(-2j*np.pi * (w0+w1))
                    res[t] += X[idx[r]] * twiddle
            for t in range(R):
                X[idx[t]] = res[t]
        # print(f'X = {X}')
    return X

R_seq = [2,3,5,4]
N = np.prod(R_seq)             
np.random.seed(0)
x0 = np.random.randn(N) + 1j*np.random.randn(N)
x1 = np.copy(x0)
x1 = fft(x1, R_seq)
print(np.linalg.norm(np.fft.fft(x0) - x1))


2.0989176627407186e-13


# Out-of-Place FFT

In [3]:
#fft 
import numpy as np
R_seq = [2,2,2,2]
N=1
for R in R_seq: N *= R
np.random.seed(0)
y = np.random.randn(N) + 1j*np.random.randn(N)
rb = np.zeros(N,dtype=np.complex128)
wb = np.zeros(N,dtype=np.complex128)
rb[:] = y[:]
np.set_printoptions(precision=1)
P = N
for R in R_seq:
    P //= R
    T = N//R
    for i in range(T):
        k,s = i//P, i%P
        src = [P*R*k + s + P*r for r in range(R)]
        dst = [i + t*N//R      for t in range(R)]
        # print(f'{i} : {src} -> {dst}')
        for t in range(R):
            wb[dst[t]] = 0
            for r in range(R):
                rot = np.exp(-2j*np.pi*r*t/R)
                twiddle = np.exp(-2j*np.pi/N*(r*k*P)) * rot
                wb[dst[t]] += rb[src[r]] * twiddle
    # print(f'wb = {wb}')
    rb,wb = wb,rb


print(np.linalg.norm(np.fft.fft(y) - rb))

6.2884604127015255e-15


# Embedded FFT

In [4]:
def factors(N):
    factor_seq = []
    f = 2
    while N % f == 0:
        factor_seq.append(f)
        N //= f
    f = 3
    while f*f <= N:
        while N % f == 0:
            factor_seq.append(f)
            N //= f
        f += 2
    if N != 1:
        factor_seq.append(N)
    return factor_seq


In [5]:
def fft_internal_outplace(y,k_,N_,P_,R_):
    N = R_
    R_seq = factors(N)
    P = N
    rb = [0] * N
    wb = [0] * N
    rb[:] = y[:]
    for R in R_seq:
        P //= R
        S = N//R
        for i in range(S):
            k,p = i//P, i%P
            src = [P*R*k + p + P*r for r in range(R)]
            dst = [i + t*N//R      for t in range(R)]
            for t in range(R):
                wb[dst[t]] = 0
                for r in range(R):
                    phi = r*k*P/N + r*t/R + k_*P_*r*P/N_
                    wb[dst[t]] += rb[src[r]] * np.exp(-2j*np.pi * phi)
        rb,wb = wb,rb
    return rb

def fft_internal_inplace(y,k_,N_,P_,R_):
    N = R_
    R_seq = factors(N)
    perm = permute_indices(N,R_seq)
    Y = [0] * N
    for i in range(N):
        Y[perm[i]] = y[i]

    P = N
    for R in R_seq:
        P //= R
        T = N//R
        for i in range(T):
            k,p = i//P, i%P
            idx = [k + r * N//(P*R) + p * N//P for r in range(R)]
            Z = [0] * R
            for t in range(R):
                for r in range(R):
                    phi = r*k*P/N + r*t/R + k_*P_*r*P/N_
                    Z[t] += Y[idx[r]] * np.exp(-2j*np.pi * phi)
            for t in range(R):
                Y[idx[t]] = Z[t]
    return Y

Choose One

In [6]:
fft_internal = fft_internal_outplace

In [7]:
fft_internal = fft_internal_inplace

In [8]:
import numpy as np

def fft_inplace(x,R_seq):
    N= len(x)
    P = N
    X = np.zeros(N,dtype=np.complex128)

    # index reordering
    perm = permute_indices(N,R_seq)
    for i in range(N):
        X[perm[i]] = x[i]

    P = N
    for R in R_seq:
        P //= R
        T = N//R
        for i in range(T):
            k,p = i//P, i%P
            idx = [k + r * N//(P*R) + p * N//P for r in range(R)]
            Y = fft_internal([ X[idx[t]] for t in range(R) ],k,N,P,R)
            for t in range(R):
                X[idx[t]] = Y[t]
    return X

def fft_outplace(x,R_seq):
    N=len(x)
    P = N
    rb = [0] * N
    wb = [0] * N
    rb[:] = x[:]
    for R in R_seq:
        P //= R
        S = N//R

        for i in range(S):
            k,s = i//P, i%P
            src = [P*R*k + s + P*r for r in range(R)]
            dst = [i + t*N//R      for t in range(R)]
            Y = fft_internal([rb[r] for r in src],k,N,P,R)
            for t in range(R):
                wb[dst[t]] = Y[t]
        rb,wb = wb,rb
    return rb

R_seq = [2,4]
N=1
for R in R_seq: N *= R
x = np.random.randn(N) + 1j*np.random.randn(N)

print(np.linalg.norm(np.fft.fft(x) - fft_inplace(x,R_seq)))
print(np.linalg.norm(np.fft.fft(x) - fft_outplace(x,R_seq)))

2.5799251709695548e-15
2.5799251709695548e-15


# Symmetric Pass Opt

In [9]:
import numpy as np

def conjmul(a,b):
    v0 = a.real * b.real
    v1 = a.imag * b.imag
    v2 = a.real * b.imag
    v3 = a.imag * b.real
    return v0 - v1 + (v2 + v3) * 1j, v0 + v1 + (-v2 + v3) * 1j

def fft(x, R_seq):
    X = np.zeros_like(x, dtype=np.complex128)
    N = len(X)

    idx = permute_indices(N, R_seq)
    for i in range(N):
        X[idx[i]] = x[i]
                
    P = N
    for R in R_seq:
        P //= R
        T = N // R
        for i in range(T):
            k, s = i // P, i % P
            idx = [k + r * N//(P*R) + s * N//P for r in range(R)]
            vals = [X[src] for src in idx]
            for r in range(1,R):
                phi = (r*k*P)/N
                vals[r] *= np.exp(-2j*np.pi * phi)
            X[idx[0]] = np.sum(vals)
            for t in range(1, R//2+1):
                y_0 = vals[0]
                y_1 = vals[0]
                for r in range(1, R):
                    phi = r*t/R
                    twiddle = np.exp(-2j*np.pi * phi)
                    val = vals[r]
                    m_0,m_1 = conjmul(val, twiddle)
                    y_0 += m_0
                    y_1 += m_1
                X[idx[t]] = y_0
                X[idx[R-t]] = y_1

    return X

R_seq = [5,5,5,5]
N = np.prod(R_seq)             
np.random.seed(0)
x0 = np.random.randn(N) + 1j*np.random.randn(N)
x1 = np.copy(x0)
x1 = fft(x1, R_seq)
print(np.linalg.norm(np.fft.fft(x0) - x1))

4.900974636389323e-13


In [10]:
import numpy as np

def fft(x, R_seq):
    X = np.zeros_like(x, dtype=np.complex128)
    N = len(X)

    idx = permute_indices(N, R_seq)
    for i in range(N):
        X[idx[i]] = x[i]
                
    P = N
    for R in R_seq:
        P //= R
        T = N // R
        for i in range(T):
            k, s = i // P, i % P
            idx = [k + r * N//(P*R) + s * N//P for r in range(R)]
            vals = [X[src] for src in idx]
            for r in range(1,R):
                phi = (r*k*P)/N
                vals[r] *= np.exp(-2j*np.pi * phi)
            X[idx[0]] = np.sum(vals)
            for t in range(1, R//2+1):
                y1 = vals[0]
                y2 = vals[0]
                for r in range(1, R//2+1):
                    phi = r*t/R
                    twiddle = np.exp(-2j*np.pi * phi)
                    x1 = vals[r]
                    x2 = vals[R-r]
                    # y1 += x1 * twiddle + x2 * np.conj(twiddle)
                    # y2 += x1 * np.conj(twiddle) + x2 * twiddle
                    m1 = twiddle.real * (x1+x2)
                    m2 = twiddle.imag * (x1-x2) * 1j
                    y1 += m1 + m2
                    y2 += m1 - m2
                X[idx[t]] = y1
                X[idx[R-t]] = y2

    return X

R_seq = [3]
N = np.prod(R_seq)             
np.random.seed(0)
x0 = np.random.randn(N) + 1j*np.random.randn(N)
x1 = np.copy(x0)
x1 = fft(x1, R_seq)
print(np.linalg.norm(np.fft.fft(x0) - x1))

9.155133597044475e-16
